# single-userコンテナ起動確認

## 概要

seleniumを利用し、ユーザのログイン～single-user notebook serverコンテナ起動の確認を行います。  

* 前提
    * Moodleにアクセスできること
    * Moodleにテスト用ユーザが登録済みであること
    * Jupyterhubのセットアップが完了しており、MoodleからLTI連携を用いてログインできること
    * 処理を行うDockerコンテナがMoodle, Jupyterhubにアクセス可能であること
* 実行方法
    * 各セルの説明に従って、実行してください
* 活用方法
    * 指定したユーザ分のコンテナが同時起動可能であることを確認できます。
    * 「801-資源消費量可視化.ipynb」ノートブックを利用して、動作確認中の資源消費量を確認できます。

### UnitGroup名

JupyterHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'JupyterHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

JupyterHubシステムに関する各種設定項目を指定します。

ここで設定された項目はansible変数として保存され、ファイルのmanagerノードへの配置時に各ファイルに反映されます。

### 必須設定項目

seleniumコンテナを実行するノード('manager'もしくはIPアドレスが指定可能)

In [ ]:
target_node = 'manager'

selenium実行用の資材等を配置するディレクトリ

In [ ]:
work_dir = '/srv/jupyterhub/spawntest'

lmsに設定したJupyterhubアクセス用のツールのID  
Moodleの場合、JupyterhubにアクセスするURLのクエリパラメータに `id=2` のような形で指定されている。

In [ ]:
lms_tool_id = 

### テスト用ユーザの指定

テスト用ユーザの情報を指定します。  
`edit/`ディレクトリに対象ファイルを保存してください。  
現在使用可能な形式は、`.csv`, `.yaml` です。  
以下のセルを利用し、ファイルを準備してください。

`.csv`で指定する場合のサンプルファイルをコピーします。  
このサンプルファイルを利用するか、以降の`.yaml`形式での設定用のセルを利用して、アカウント情報を登録してください。

In [ ]:
!mkdir -p edit
!cp ../tools/selenium/selenium-client/accounts_sample.csv edit/accounts_sample.csv

アカウント情報を記載するファイル名を指定してください。  
以降のセルを利用して設定する場合は、変更不要です。

In [ ]:
# accounts_file = 'accounts_sample.csv'
accounts_file = 'accounts_file.yaml'

以下の`params`の内容を編集し、アカウント情報を登録してください。  
先に用意した`edit/accounts_sample.csv`や、自身で用意したファイルを利用する場合は、以下のセルを使用する必要はありません。

In [ ]:
import yaml

path = f'edit/accounts_file.yaml'
params = [
    {'username': 'admin', 'password': 'adminpass01'},
]

with open(path, encoding='utf-8', mode="w") as f:
    yaml.safe_dump(params, stream=f)


### パラメータの保存

ansibleで利用するために、パラメータを対象ファイルに出力します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)

gvars.update({
    'lms_tool_id': lms_tool_id,
    'accounts_file': accounts_file,
})

with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

### Selenium

Selenium実行・処理コンテナに関するセットアップを行った後、実行します。

テスト対象のサーバにseleniumがアクセスできることを確認します。

In [ ]:
# selenium実行環境からmoodleへのアクセス
!ansible {target_hub} -m shell \
    -a "curl {{{{lms_platform_id}}}} -o /dev/null -w '%{{http_code}}\n' -s"
# selenium実行環境からjupyterhubへのアクセス
!ansible {target_hub} -m shell \
    -a "curl https://{{{{jupyterhub_fqdn}}}}/hub/health -o /dev/null -w '%{{http_code}}\n' -s"

#### セットアップ

Selenium実行・処理コンテナに関するセットアップを行います。

まず、ファイルを格納するディレクトリを作成

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path={work_dir} state=directory owner={{{{ansible_user}}}}'

ファイルの配置をチェックモードで試行

In [ ]:
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=../tools/selenium dest={work_dir}'

!ansible {target_hub} -CDv -m synchronize \
    -a 'src=edit/{accounts_file} dest={work_dir}/selenium/{accounts_file}' || true

!ansible {target_hub} -CDv -m template \
    -a 'src=../tools/selenium/docker-compose.yml dest={work_dir}/selenium backup=yes' || true

実際にファイルを配置

In [ ]:
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=../tools/selenium dest={work_dir}'

!ansible {target_hub} -Dv -m synchronize \
    -a 'src=edit/{accounts_file} dest={work_dir}/selenium/{accounts_file}'

!ansible {target_hub} -Dv -m template \
    -a 'src=../tools/selenium/docker-compose.yml dest={work_dir}/selenium backup=yes'

#### 実行

In [ ]:
import time

# コンテナを起動
!ansible {target_hub} -m shell \
    -a 'chdir={work_dir}/selenium/ docker-compose up -d --build'

実行完了まで待機します。

In [ ]:
MAX_RETRY = 30
INTERVAL = 10
for i in range(MAX_RETRY):
    active_user = !ansible {target_hub} -m shell \
        -a 'docker ps | grep -c selenium-client || true'
    if int(active_user[1]) == 0:
        print('Selenium client finished.')
        break
    
    print('Waiting selenium finish.')
    time.sleep(INTERVAL)
else:
    raise Exception(f'Not finished in {INTERVAL*MAX_RETRY} secondes.')

結果を確認します。  
以下のセルを実行すると、selenium実行結果ファイルのうち最新のものの内容を表示します。

In [ ]:
!ansible {target_hub} -m shell \
    -a 'ls -rt {work_dir}/selenium/selenium-client/result/* | tail -n 1 | xargs cat'

結果が出力されない場合、アクセス自体出来ていない可能性があります。  
そのような場合、以下のセルを実行し、selenium実行コンテナのログを確認します。  
「Failed to connect」等のログがある場合、seleniumを実行している環境からLMSへのアクセス許可等の確認を行ってください。

In [ ]:
!ansible {target_hub} -m shell \
    -a 'docker logs selenium-executer'

#### 終了  

コンテナをシャットダウンします。

In [ ]:
!ansible {target_hub} -m shell \
    -a 'chdir={work_dir}/selenium/ docker-compose down'
!ansible {target_hub} -m shell \
    -a 'chdir={work_dir}/selenium/ docker-compose rm'